In [15]:
from pymatgen.analysis.bond_valence import BVAnalyzer 
from pymatgen.core.structure import IStructure
from monty.serialization import loadfn
from pymatgen.core.periodic_table import Element, Species, get_el_sp

def cal_BVS(i, mid, path):
    OS = BVAnalyzer()
    try:
        struct = IStructure.from_file(path + mid)
        print("Get the %s-th structre of the name %s."%(i, mid))
        valence_list = OS.get_valences(struct)
        print("Get the %s-th valences of the name %s successfully."%(i, mid))
    except Exception as e:
        print(f"An error occurred: {e}")
        ele_list = list(map(lambda x:x.specie.name, struct.sites))
        if len(set(ele_list)) == 1:
            valence_list = [0] * len(ele_list)
        else:
            valence_list = None

    parameters = [mid, valence_list]
    return parameters

In [16]:
para = cal_BVS(0, "mp-31770.cif", "D:/share/TOSS_2024/structures/")

Get the 0-th structre of the name mp-31770.cif.
Get the 0-th valences of the name mp-31770.cif successfully.


In [79]:
BV_2020 = []
import pandas as pd
column_names = ['ele1','vl1','ele2','vl2','Ro','B','ref_id', 'details']
BV_2020 = df = pd.read_csv("C:/Users/YY/Downloads/bvparm2020.cif", delim_whitespace=True, names=column_names, skiprows=180)

In [92]:
BV_dict_2020 = {}

for index, row in BV_2020.iterrows():
    spec1 = (row["ele1"],row["vl1"])
    spec2 = (row["ele2"],row["vl2"])
    key = (spec1, spec2)
    if key not in BV_dict_2020:
        BV_dict_2020[key] = []
    BV_dict_2020[key].append((row["Ro"], row["B"]))
    
print(len(BV_dict_2020))
print(BV_2020.shape)

1369
(1935, 8)


In [116]:
for key, RoB in BV_dict_2020.items():
    ele_pair = (key[0][0], key[1][0])
    if ele_pair == ("Fe","O") or ele_pair == ("O","Fe"):
        print("good")



good
good
good
good


In [136]:
from math import exp, sqrt
import numpy as np

BV_PARAMS = {}
for k, v in loadfn("D:/Anaconda3/envs/GFOS/Lib/site-packages/pymatgen/analysis/bvparam_1991.yaml").items():
    BV_PARAMS[Element(k)] = v

all_data = loadfn("D:/Anaconda3/envs/GFOS/Lib/site-packages/pymatgen/analysis/icsd_bv.yaml")

ICSD_BV_DATA = {Species.from_string(sp): data for sp, data in all_data["bvsum"].items()}
PRIOR_PROB = {Species.from_string(sp): data for sp, data in all_data["occurrence"].items()}

icsd_bv_data = ({get_el_sp(specie): data for specie, data in ICSD_BV_DATA.items()} if len(forbidden_species) > 0 else ICSD_BV_DATA)

ELECTRONEG = [
    Element(sym)
    for sym in ["H","B","C","Si","N","P","As","Sb","O","S","Se","Te","F","Cl","Br","I"]]


def calculate_bv_sum_1991(site, nn_list, scale_factor=1.0):
    el1 = Element(site.specie.symbol)
    bvsum = 0
    for nn in nn_list:
        el2 = Element(nn.specie.symbol)
        if (el1 in ELECTRONEG or el2 in ELECTRONEG) and el1 != el2:
            r1 = BV_PARAMS[el1]["r"]
            r2 = BV_PARAMS[el2]["r"]
            c1 = BV_PARAMS[el1]["c"]
            c2 = BV_PARAMS[el2]["c"]
            R = r1 + r2 - r1 * r2 * (sqrt(c1) - sqrt(c2)) ** 2 / (c1 * r1 + c2 * r2)
            vij = exp((R - nn.nn_distance * scale_factor) / 0.31)
            bvsum += vij * (1 if el1.X < el2.X else -1)
    return bvsum


def calculate_bv_sum_2020(site, nn_list, scale_factor=1.0):
    el1 = Element(site.specie.symbol)
    bvsum = 0
    for nn in nn_list:
        el2 = Element(nn.specie.symbol)
        ###
        bvsum_candi = []
        for key, RoB in BV_dict_2020.items():
            ele_pair = (key[0][0], key[1][0])
            if ele_pair == (el1.symbol,el2.symbol) or ele_pair == (el2.symbol,el1.symbol):
                for (Ro,B) in RoB:
                    Ro = np.array(Ro, dtype="float32")
                    B = np.array(B, dtype="float32")
                    vij = exp((Ro - nn.nn_distance * scale_factor) / B)
                    bvsum += vij * (1 if el1.X < el2.X else -1)
                    bvsum_candi.append(bvsum)
        print(bvsum_candi)
        ###
        if (el1 in ELECTRONEG or el2 in ELECTRONEG) and el1 != el2:
            print(el1,el2)
            r1 = BV_PARAMS[el1]["r"]
            r2 = BV_PARAMS[el2]["r"]
            c1 = BV_PARAMS[el1]["c"]
            c2 = BV_PARAMS[el2]["c"]
            R = r1 + r2 - r1 * r2 * (sqrt(c1) - sqrt(c2)) ** 2 / (c1 * r1 + c2 * r2)
            vij = exp((R - nn.nn_distance * scale_factor) / 0.31)
            bvsum += vij * (1 if el1.X < el2.X else -1)
    return bvsum


def _calc_site_probabilities(site, nn):
    el = site.specie.symbol
    bv_sum = calculate_bv_sum_2020(site, nn, scale_factor=1.015)
    print(bv_sum)
    prob = {}
    for sp, data in icsd_bv_data.items():  #The sp contains the OS now.
        if sp.symbol == el and sp.oxi_state != 0 and data["std"] > 0:
            u = data["mean"]
            sigma = data["std"]
            prob[sp.oxi_state] = exp(-((bv_sum - u) ** 2) / 2 / (sigma ** 2)) / sigma * PRIOR_PROB[sp]
    try:
        prob = {k: v / sum(prob.values()) for k, v in prob.items()}
    except ZeroDivisionError:
        prob = {k: 0.0 for k in prob}
    return prob


def get_valences(structure):
    els = [Element(el.symbol) for el in structure.composition.elements]

    equi_sites = [[site] for site in structure]

    equi_sites = sorted(equi_sites, key=lambda sites: -sites[0].species.average_electroneg)

    valences = []
    all_prob = []
    nns = []
    if structure.is_ordered:
        for sites in equi_sites:
            test_site = sites[0]
            nn = structure.get_neighbors(test_site, 4)
            nns.append(len(nn))
            prob = _calc_site_probabilities(test_site, nn)
            all_prob.append(prob)
            val = list(prob.keys())
            # Sort valences in order of decreasing probability.
            val = sorted(val, key=lambda v: -prob[v])
            # Retain probabilities that are at least 1/100 of highest prob.
            valences.append(list(filter(lambda v: prob[v] > 0.01 * prob[val[0]], val)))
    return valences

In [137]:
mid = "mp-31770.cif"
path = "D:/share/TOSS_2024/structures/"

struct = IStructure.from_file(path + mid)

valences = get_valences(struct)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[15.923554029006196, 16.21047445285457, 16.467203261907525, 16.7150684853194, 17.005783751350574, 17.29220742010882, 17.57670449670811, 17.87217252932191, 18.14384756937291, 18.389540337336367, 18.675215473463204]
[15.923554029006196, 16.21047445285457, 16.467203261907525, 16.7150684853194, 17.005783751350574, 17.29220742010882, 17.57670449670811, 17.87217252932191, 18.14384756937291, 18.389540337336367, 18.675215473463204]
[15.923554029006196, 16.21047445285457, 16.467203261907525, 16.7150684853194, 17.005783751350574, 17.29220742010882, 17.57670449670811, 17.87217252932191, 18.14384756937291, 18.389540337336367, 18.675215473463204]
[15.923554029006196, 16.21047445285457, 16.467203261907525, 16.7150684853194, 17.005783751350574, 17.29220742010882, 17.57670449670811, 17.87217252932191, 18.14384756937291, 18.389540337336367, 18.675215473463204]
[15.923554029006196, 16.21047445285457, 16.467203261907525, 16.7150684853194, 17.005783751350574, 17.29220742010882, 17.57670449670811, 17.87217

In [55]:
icsd_bv_data

{Species Ag+: {'mean': 0.7656466923357081,
  'n_data_pts': 964,
  'std': 0.4136944016489287},
 Species Ag0+: {'mean': 0.8458145349421629,
  'n_data_pts': 57,
  'std': 0.5234818161615782},
 Species Ag2+: {'mean': 0.9355055821858393,
  'n_data_pts': 32,
  'std': 0.1562431976543218},
 Species Ag3+: {'mean': 1.3783868963043135,
  'n_data_pts': 17,
  'std': 0.49597991838957073},
 Species Al0+: {'mean': 1.3316931867351518,
  'n_data_pts': 229,
  'std': 1.1224575121673033},
 Species Al2+: {'mean': 1.5739365543187454,
  'n_data_pts': 2,
  'std': 0.0019450548385169242},
 Species Al3+: {'mean': 3.254336879371157,
  'n_data_pts': 1150,
  'std': 0.6833210681705788},
 Species As-: {'mean': -1.3221551731343792,
  'n_data_pts': 36,
  'std': 0.9200676257870635},
 Species As0+: {'mean': -2.090630464780604,
  'n_data_pts': 295,
  'std': 1.9142182924572342},
 Species As2+: {'mean': 1.399095314328814,
  'n_data_pts': 28,
  'std': 1.542341395277636},
 Species As2-: {'mean': -2.5548918780763974,
  'n_data_p

In [139]:
160*200*3/60/60

26.666666666666668